In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [2]:
## importing the dataset

df= pd.read_csv('stud.csv')
df.head()

,gender,race_ethnicity,parental_level_of_education,lunch,test_preparation_course,math_score,reading_score,writing_score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


- we need to encode the categorical data using one-hot encoder
- we need to standardize the dataset for training machine learning model

In [4]:
# splitting the independent and the dependent variables

X= df.drop(columns=['math_score'], axis=1)
y= df['math_score']

In [10]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

num_feature= X.select_dtypes(exclude="object").columns
cat_feature= X.select_dtypes(include="object").columns

print(num_feature)
print(cat_feature)

Index(['reading_score', 'writing_score'], dtype='object')
Index(['gender', 'race_ethnicity', 'parental_level_of_education', 'lunch',
       'test_preparation_course'],
      dtype='object')


In [13]:
# Initializing the functions

oh_encoder= OneHotEncoder()
standard= StandardScaler()

preprocessor = ColumnTransformer(
    [
        ('OneHotEncoder', oh_encoder, cat_feature),
        ('StandardScaler', standard, num_feature)
    ]
)


In [14]:
## Transforming the dataset

X= preprocessor.fit_transform(X)

In [15]:
X.shape

(1000, 19)

In [17]:
X

array([[ 1.        ,  0.        ,  0.        , ...,  1.        ,
         0.19399858,  0.39149181],
       [ 1.        ,  0.        ,  0.        , ...,  0.        ,
         1.42747598,  1.31326868],
       [ 1.        ,  0.        ,  0.        , ...,  1.        ,
         1.77010859,  1.64247471],
       ...,
       [ 1.        ,  0.        ,  0.        , ...,  0.        ,
         0.12547206, -0.20107904],
       [ 1.        ,  0.        ,  0.        , ...,  0.        ,
         0.60515772,  0.58901542],
       [ 1.        ,  0.        ,  0.        , ...,  1.        ,
         1.15336989,  1.18158627]])

In [20]:
## splitting the dataset
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test= train_test_split(X,y, test_size=0.35, random_state=45)

In [21]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((650, 19), (350, 19), (650,), (350,))

In [35]:
!pip install xgboost

  Using cached xgboost-3.0.0-py3-none-manylinux_2_28_x86_64.whl.metadata (2.1 kB)
  Using cached nvidia_nccl_cu12-2.26.2.post1-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (2.0 kB)
Using cached xgboost-3.0.0-py3-none-manylinux_2_28_x86_64.whl (253.9 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╺━━━━━ 249.0/291.7 MB 1.2 MB/s eta 0:00:36m�━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/291.7 MB 690.1 kB/s eta 0:06:49��━━━━╺━━━━━━━━━━━━━━━━━━━━━━ 123.2/291.7 MB 938.6 kB/s eta 0:03:00━━━━━━━━━━━━━━━━━━ 125.6/291.7 MB 972.6 kB/s eta 0:02:51�━━ 127.7/291.7 MB 991.7 kB/s eta 0:02:46�━━ 181.9/291.7 MB 604.6 kB/s eta 0:03:02kB/s eta 0:03:14�━━━━━━━━━━━━━━━━━━━━━━╸━━━━━━━━━━━━━━ 184.0/291.7 MB 490.2 kB/s eta 0:03:40━━━━━━━━━━━━ 184.5/291.7 MB 402.4 kB/s eta 0:04:27━━━━━━ 187.7/291.7 MB 288.1 kB/s eta 0:06:01ERROR: Could not install packages due to an OSError: [Errno 28] No space left on device

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╺━━━━━ 249.0/291.7 MB 1.2 MB/s eta 0:00:36


In [25]:
## importing the diffenret models to use

from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.svm import SVR
from sklearn.ensemble import AdaBoostRegressor, GradientBoostingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from catboost import CatBoostRegressor
import math

In [28]:
## creating the function for metrics

def model_evaluate(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse= mean_squared_error(true, predicted)
    rmse= math.sqrt(mse)
    r2score= r2_score(true, predicted)

    return mae, mse, rmse, r2score

In [33]:
#training the model

models= {
    'Linear Regressor': LinearRegression(),
    'Ridge': Ridge(),
    'Lasso': Lasso(),
    'SVR': SVR(),
    'KNeighbors Regressor': KNeighborsRegressor(),
    'CatBoost Regressor': CatBoostRegressor(),
    'DecisionTreeRegressor': DecisionTreeRegressor(),
    'AdaBoost Regressor': AdaBoostRegressor(),
    'Gradient Boosting Regressor': GradientBoostingRegressor()
}

# loops for training the model

model_list= []
r2score_list=[]

for i in range(len(list(models))):
    model= list(models.values())[i]
    model.fit(X_train, y_train)

    ## Evaluating the model
    y_train_pred= model.predict(X_train)
    y_test_pred= model.predict(X_test)

    train_mae, train_mse, train_rmse, train_r2score= model_evaluate(y_train, y_train_pred)
    test_mae, test_mse, test_rmse, test_r2score= model_evaluate(y_test, y_test_pred)


    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])

    print('Model performance for Training set')
    print("- Mean Squared Error: {:.4f}".format(train_mse))
    print("- Root Mean Squared Error: {:.4f}".format(train_rmse))
    print("- Mean Absolute Error: {:.4f}".format(train_mae))
    print("- R2 Score: {:.4f}".format(train_r2score))

    print('----------------------------------')
    
    print('Model performance for Test set')
    print("- Mean Squared Error: {:.4f}".format(test_mse))
    print("- Root Mean Squared Error: {:.4f}".format(test_rmse))
    print("- Mean Absolute Error: {:.4f}".format(test_mae))
    print("- R2 Score: {:.4f}".format(test_r2score))
    r2score_list.append(test_r2score)
    
    print('='*35)
    print('\n')

    


Linear Regressor
Model performance for Training set
- Mean Squared Error: 27.2386
- Root Mean Squared Error: 5.2191
- Mean Absolute Error: 4.1357
- R2 Score: 0.8812
----------------------------------
Model performance for Test set
- Mean Squared Error: 30.8998
- Root Mean Squared Error: 5.5588
- Mean Absolute Error: 4.4597
- R2 Score: 0.8626


Ridge
Model performance for Training set
- Mean Squared Error: 27.2419
- Root Mean Squared Error: 5.2194
- Mean Absolute Error: 4.1355
- R2 Score: 0.8812
----------------------------------
Model performance for Test set
- Mean Squared Error: 30.9463
- Root Mean Squared Error: 5.5629
- Mean Absolute Error: 4.4626
- R2 Score: 0.8624


Lasso
Model performance for Training set
- Mean Squared Error: 41.3398
- Root Mean Squared Error: 6.4296
- Mean Absolute Error: 5.0602
- R2 Score: 0.8197
----------------------------------
Model performance for Test set
- Mean Squared Error: 48.4437
- Root Mean Squared Error: 6.9601
- Mean Absolute Error: 5.5663
- R2 

In [34]:
pd.DataFrame(list(zip(model_list, r2score_list)), 
             columns=['Model Name', 'R2_Score']).sort_values(by=["R2_Score"],ascending=False)

,Model Name,R2_Score
0,Linear Regressor,0.862567
1,Ridge,0.862360
8,Gradient Boosting Regressor,0.840556
5,CatBoost Regressor,0.834439
7,AdaBoost Regressor,0.798997
2,Lasso,0.784537
4,KNeighbors Regressor,0.770564
3,SVR,0.727950
6,DecisionTreeRegressor,0.679689
